# TP 2025-2026 — Intrusion & DDoS Detection using AI
**Auteur :** TCHOMO KOMBOU THIERRY ARMEL

In [3]:
#Imports & dossiers
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, roc_auc_score, roc_curve, classification_report,
                             precision_recall_curve, auc)


from sklearn.decomposition import PCA


OUT = Path('output')
FIG = OUT / 'figures'
OUT.mkdir(exist_ok=True)
FIG.mkdir(exist_ok=True)
print("Output folder:", OUT.resolve())


Output folder: C:\Users\thier\Desktop\TPs\TP_1_5CS08_TP\output


In [4]:
#Chargement
TRAIN = 'training_dataset_CIC_DDoS_2019.csv'
TEST  = 'testing_dataset_CIC_DDoS_2019.csv'

df_train = pd.read_csv(TRAIN)
df_test  = pd.read_csv(TEST)
print("Train shape:", df_train.shape)
print("Test shape: ", df_test.shape)

Train shape: (488041, 86)
Test shape:  (110399, 86)


In [5]:
#EDA complet
# 1) shape / dtypes
print(df_train.info())

# 2) Target / Label distributions
print("\nTarget counts (train):")
print(df_train['Target'].value_counts(dropna=False))
print("\nLabel sample counts (train):")
print(df_train['Label'].value_counts().head(20))

# 3) Missing values
missing = df_train.isnull().sum().sort_values(ascending=False)
print("\nTop missing (train):")
print(missing[missing>0].head(30))
missing.to_csv(OUT/'missing_counts_train.csv')

# 4) Constant columns
const_cols = [c for c in df_train.columns if df_train[c].nunique()==1]
print("\nConstant columns:", const_cols)

# 5) ID-like columns
id_like = [c for c in df_train.columns if any(k in c.lower() for k in ['flow','ip','timestamp','id','src','dst'])]
print("\nPotential ID-like columns:", id_like)

# 6) Duplicates
print("\nNumber of duplicate rows (train):", df_train.duplicated().sum())

# 7) Numeric correlation & highly correlated pairs
num = df_train.select_dtypes(include=['int64','float64']).apply(pd.to_numeric, errors='coerce')
cor = num.corr().abs()
pairs = []
for i in range(len(cor.columns)):
    for j in range(i+1, len(cor.columns)):
        v = cor.iloc[i,j]
        if pd.notna(v) and v > 0.95:
            pairs.append((cor.columns[i], cor.columns[j], v))
print("\nHighly correlated pairs (>0.95):", pairs[:30])

# 8) Save correlation heatmap (subset)
cols = num.columns[:30]
plt.figure(figsize=(12,10))
sns.heatmap(num[cols].corr(), cmap='viridis')
plt.title('Correlation heatmap (first 30 numeric features)')
plt.tight_layout()
plt.savefig(FIG/'corr_heatmap_subset.png', dpi=200)
plt.close()
print("Saved corr_heatmap_subset.png")

# 9) Target distribution plot
plt.figure()
df_train['Target'].value_counts().plot(kind='bar')
plt.title('Target distribution (train)')
plt.tight_layout()
plt.savefig(FIG/'target_distribution.png', dpi=200)
plt.close()
print("Saved target_distribution.png")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488041 entries, 0 to 488040
Data columns (total 86 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Flow ID                      488041 non-null  object 
 1   Source IP                    488041 non-null  object 
 2   Source Port                  488041 non-null  int64  
 3   Destination IP               488041 non-null  object 
 4   Destination Port             488041 non-null  int64  
 5   Protocol                     488041 non-null  int64  
 6   Timestamp                    488041 non-null  object 
 7   Flow Duration                488041 non-null  int64  
 8   Total Fwd Packets            488041 non-null  int64  
 9   Total Backward Packets       488041 non-null  int64  
 10  Total Length of Fwd Packets  488041 non-null  float64
 11  Total Length of Bwd Packets  488041 non-null  float64
 12  Fwd Packet Length Max        488041 non-null  float64
 13 

In [6]:
#EDA

CONST_COLS = const_cols          # colonnes constantes déjà trouvées
ID_COLS    = id_like             # colonnes identifiantes déjà trouvées
HIGH_CORR_PAIRS = pairs          # liste de tuples (col1, col2, corr)

print("CONST_COLS :", CONST_COLS)
print("ID_COLS    :", ID_COLS)
print("Nb de paires corrélées (>0.95) :", len(HIGH_CORR_PAIRS))

# Pour chaque paire (a, b, corr), on garde a et on supprime b
to_drop_corr = set()
for a, b, v in HIGH_CORR_PAIRS:
    # on décide de toujours supprimer le deuxième de la paire
    to_drop_corr.add(b)

TO_DROP_CORR = list(to_drop_corr)
print("Nombre de colonnes à supprimer pour corrélation :", len(TO_DROP_CORR))
print("Exemples de colonnes supprimées (corr élevées) :", TO_DROP_CORR[:15])

# 2) Fonction qui nettoie : Target -> binaire, drop ID + constantes, encodage Protocol, conversion numérique
def clean_base_from_eda(df):
    df2 = df.copy()
    # Cible binaire
    df2['Target'] = df2['Target'].astype(str).map({'DDoS': 1, 'Benign': 0}).fillna(0).astype(int)
    
    # Drop identifiants + colonnes constantes
    df2 = df2.drop(columns=ID_COLS + CONST_COLS, errors='ignore')
    
    y = df2['Target'].astype(int)
    X = df2.drop(columns=['Label', 'Target'], errors=True)
    
    # Encodage de Protocol si présent
    if 'Protocol' in X.columns:
        X['Protocol'] = X['Protocol'].astype(str)
        X = pd.get_dummies(X, columns=['Protocol'], drop_first=True)
    
    # Tout en numérique + imputation simple
    X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
    return X, y

# Application à train / test
X_base, y_full = clean_base_from_eda(df_train)
X_test_base, y_test_full = clean_base_from_eda(df_test)

print("Shape base train (après drop ID + constantes):", X_base.shape)
print("Shape base test  (après drop ID + constantes):", X_test_base.shape)

# 3) Construction des jeux de features par modèle

# RF : on garde toutes les colonnes restantes (corrélées incluses)
X_rf = X_base.copy()

# KNN : on supprime toutes les colonnes fortement corrélées
X_knn = X_base.drop(columns=TO_DROP_CORR, errors='ignore')

# AdaBoost : même logique que KNN (suppression corrélées), sans PCA ensuite
X_ada = X_base.drop(columns=TO_DROP_CORR, errors='ignore')

print("Shapes après filtrage corrélations :")
print("  X_rf  :", X_rf.shape)
print("  X_knn :", X_knn.shape)
print("  X_ada :", X_ada.shape)

# 4) Split train / validation (même y, même random_state pour cohérence)
X_rf_train, X_rf_val, y_rf_train, y_rf_val = train_test_split(
    X_rf, y_full, test_size=0.3, random_state=42, shuffle=True, stratify=y_full
)
X_knn_train, X_knn_val, y_knn_train, y_knn_val = train_test_split(
    X_knn, y_full, test_size=0.3, random_state=42, shuffle=True, stratify=y_full
)
X_ada_train, X_ada_val, y_ada_train, y_ada_val = train_test_split(
    X_ada, y_full, test_size=0.3, random_state=42, shuffle=True, stratify=y_full
)

# Adapter X_test pour chaque jeu de colonnes
X_rf_test  = X_test_base[X_rf.columns]
X_knn_test = X_test_base[X_knn.columns]
X_ada_test = X_test_base[X_ada.columns]

print("Train/val RF  :", X_rf_train.shape, X_rf_val.shape)
print("Train/val KNN :", X_knn_train.shape, X_knn_val.shape)
print("Train/val Ada :", X_ada_train.shape, X_ada_val.shape)

# 5) StandardScaler pour les trois modèles

scaler_rf  = StandardScaler()
scaler_knn = StandardScaler()
scaler_ada = StandardScaler()

X_rf_train_scaled  = scaler_rf.fit_transform(X_rf_train)
X_rf_val_scaled    = scaler_rf.transform(X_rf_val)
X_rf_test_scaled   = scaler_rf.transform(X_rf_test)

X_knn_train_scaled = scaler_knn.fit_transform(X_knn_train)
X_knn_val_scaled   = scaler_knn.transform(X_knn_val)
X_knn_test_scaled  = scaler_knn.transform(X_knn_test)

X_ada_train_scaled = scaler_ada.fit_transform(X_ada_train)
X_ada_val_scaled   = scaler_ada.transform(X_ada_val)
X_ada_test_scaled  = scaler_ada.transform(X_ada_test)

# 6) PCA pour KNN uniquement (on garde ~95% de la variance)
pca_knn = PCA(n_components=0.95)
X_knn_train_pca = pca_knn.fit_transform(X_knn_train_scaled)
X_knn_val_pca   = pca_knn.transform(X_knn_val_scaled)
X_knn_test_pca  = pca_knn.transform(X_knn_test_scaled)

print("Dimension KNN avant PCA :", X_knn_train.shape[1])
print("Dimension KNN après PCA :", X_knn_train_pca.shape[1])

CONST_COLS : ['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'FIN Flag Count', 'PSH Flag Count', 'ECE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']
ID_COLS    : ['Flow ID', 'Source IP', 'Destination IP', 'Timestamp', 'Flow Duration', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']
Nb de paires corrélées (>0.95) : 50
Nombre de colonnes à supprimer pour corrélation : 27
Exemples de colonnes supprimées (corr élevées) : ['Fwd IAT Min', 'Subflow Bwd Bytes', 'Avg Bwd Segment Size', 'Fwd IAT Mean', 'Fwd Header Length.1', 'Average Packet Size', 'Fwd Packet Length Mean', 'Min Packet Length', 'Idle Min', 'Bwd IAT Std', 'Subflow Bwd Packets', 'RST Flag Count', 'Flow IAT Std', 'Bwd Packet Length Std', 'Idle Ma

In [ ]:
# === PARTIE C : Entraînement et évaluation des modèles (RF, KNN, AdaBoost) ===
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json

# Sécurité : s'assurer que les dossiers existent
OUT.mkdir(exist_ok=True)
FIG.mkdir(exist_ok=True)

val_results = []
test_results = []

# ========= 1) RANDOM FOREST =========
print("\n=== RANDOM FOREST ===")
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_rf_train_scaled, y_rf_train)

# Validation
y_rf_val_pred  = rf.predict(X_rf_val_scaled)
y_rf_val_proba = rf.predict_proba(X_rf_val_scaled)[:, 1]

rf_val_acc  = accuracy_score(y_rf_val, y_rf_val_pred)
rf_val_prec = precision_score(y_rf_val, y_rf_val_pred, zero_division=0)
rf_val_rec  = recall_score(y_rf_val, y_rf_val_pred, zero_division=0)
rf_val_f1   = f1_score(y_rf_val, y_rf_val_pred, zero_division=0)
rf_val_auc  = roc_auc_score(y_rf_val, y_rf_val_proba)

cm_rf_val = confusion_matrix(y_rf_val, y_rf_val_pred)

# Confusion matrix (val)
plt.figure(figsize=(4,3))
sns.heatmap(cm_rf_val, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix - RandomForest (val)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(FIG / 'confusion_RandomForest_val.png', dpi=200)
plt.close()

# ROC (val)
fpr, tpr, _ = roc_curve(y_rf_val, y_rf_val_proba)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC={rf_val_auc:.4f}')
plt.plot([0,1],[0,1],'--')
plt.title('ROC - RandomForest (val)')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.tight_layout()
plt.savefig(FIG / 'roc_RandomForest_val.png', dpi=200)
plt.close()

val_results.append({
    'model': 'RandomForest',
    'accuracy': rf_val_acc,
    'precision': rf_val_prec,
    'recall': rf_val_rec,
    'f1': rf_val_f1,
    'roc_auc': rf_val_auc
})

# Test
# Probabilités prédites pour la classe 1
y_rf_test_proba = rf.predict_proba(X_rf_test_scaled)[:, 1]

# Seuil personnalisé, par exemple 0.5
threshold = 0.5
y_rf_test_pred = (y_rf_test_proba >= threshold).astype(int)

y_rf_test_proba = rf.predict_proba(X_rf_test_scaled)[:, 1]

rf_test_acc  = accuracy_score(y_test_full, y_rf_test_pred)
rf_test_prec = precision_score(y_test_full, y_rf_test_pred, zero_division=0)
rf_test_rec  = recall_score(y_test_full, y_rf_test_pred, zero_division=0)
rf_test_f1   = f1_score(y_test_full, y_rf_test_pred, zero_division=0)
rf_test_auc  = roc_auc_score(y_test_full, y_rf_test_proba)

cm_rf_test = confusion_matrix(y_test_full, y_rf_test_pred)

plt.figure(figsize=(4,3))
sns.heatmap(cm_rf_test, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix - RandomForest (test)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(FIG / 'confusion_RandomForest_test.png', dpi=200)
plt.close()

fpr, tpr, _ = roc_curve(y_test_full, y_rf_test_proba)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC={rf_test_auc:.4f}')
plt.plot([0,1],[0,1],'--')
plt.title('ROC - RandomForest (test)')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.tight_layout()
plt.savefig(FIG / 'roc_RandomForest_test.png', dpi=200)
plt.close()

test_results.append({
    'model': 'RandomForest',
    'accuracy': rf_test_acc,
    'precision': rf_test_prec,
    'recall': rf_test_rec,
    'f1': rf_test_f1,
    'roc_auc': rf_test_auc
})

print("Classification report (test) - RandomForest :")
print(classification_report(y_test_full, y_rf_test_pred, digits=4))


# ========= 2) KNN (avec PCA) =========
print("\n=== KNN (avec PCA) ===")
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_knn_train_pca, y_knn_train)

# Validation
y_knn_val_pred  = knn.predict(X_knn_val_pca)
y_knn_val_proba = knn.predict_proba(X_knn_val_pca)[:, 1]

knn_val_acc  = accuracy_score(y_knn_val, y_knn_val_pred)
knn_val_prec = precision_score(y_knn_val, y_knn_val_pred, zero_division=0)
knn_val_rec  = recall_score(y_knn_val, y_knn_val_pred, zero_division=0)
knn_val_f1   = f1_score(y_knn_val, y_knn_val_pred, zero_division=0)
knn_val_auc  = roc_auc_score(y_knn_val, y_knn_val_proba)

cm_knn_val = confusion_matrix(y_knn_val, y_knn_val_pred)

plt.figure(figsize=(4,3))
sns.heatmap(cm_knn_val, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix - KNN (val)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(FIG / 'confusion_KNN_val.png', dpi=200)
plt.close()

fpr, tpr, _ = roc_curve(y_knn_val, y_knn_val_proba)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC={knn_val_auc:.4f}')
plt.plot([0,1],[0,1],'--')
plt.title('ROC - KNN (val)')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.tight_layout()
plt.savefig(FIG / 'roc_KNN_val.png', dpi=200)
plt.close()

val_results.append({
    'model': 'KNN',
    'accuracy': knn_val_acc,
    'precision': knn_val_prec,
    'recall': knn_val_rec,
    'f1': knn_val_f1,
    'roc_auc': knn_val_auc
})

# Test
y_knn_test_pred  = knn.predict(X_knn_test_pca)
y_knn_test_proba = knn.predict_proba(X_knn_test_pca)[:, 1]

knn_test_acc  = accuracy_score(y_test_full, y_knn_test_pred)
knn_test_prec = precision_score(y_test_full, y_knn_test_pred, zero_division=0)
knn_test_rec  = recall_score(y_test_full, y_knn_test_pred, zero_division=0)
knn_test_f1   = f1_score(y_test_full, y_knn_test_pred, zero_division=0)
knn_test_auc  = roc_auc_score(y_test_full, y_knn_test_proba)

cm_knn_test = confusion_matrix(y_test_full, y_knn_test_pred)

plt.figure(figsize=(4,3))
sns.heatmap(cm_knn_test, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix - KNN (test)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(FIG / 'confusion_KNN_test.png', dpi=200)
plt.close()

fpr, tpr, _ = roc_curve(y_test_full, y_knn_test_proba)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC={knn_test_auc:.4f}')
plt.plot([0,1],[0,1],'--')
plt.title('ROC - KNN (test)')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.tight_layout()
plt.savefig(FIG / 'roc_KNN_test.png', dpi=200)
plt.close()

test_results.append({
    'model': 'KNN',
    'accuracy': knn_test_acc,
    'precision': knn_test_prec,
    'recall': knn_test_rec,
    'f1': knn_test_f1,
    'roc_auc': knn_test_auc
})

print("Classification report (test) - KNN :")
print(classification_report(y_test_full, y_knn_test_pred, digits=4))


# ========= 3) ADABOOST =========
print("\n=== AdaBoost ===")
ada = AdaBoostClassifier(
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)

ada.fit(X_ada_train_scaled, y_ada_train)

# Validation
y_ada_val_pred  = ada.predict(X_ada_val_scaled)
y_ada_val_proba = ada.predict_proba(X_ada_val_scaled)[:, 1]

ada_val_acc  = accuracy_score(y_ada_val, y_ada_val_pred)
ada_val_prec = precision_score(y_ada_val, y_ada_val_pred, zero_division=0)
ada_val_rec  = recall_score(y_ada_val, y_ada_val_pred, zero_division=0)
ada_val_f1   = f1_score(y_ada_val, y_ada_val_pred, zero_division=0)
ada_val_auc  = roc_auc_score(y_ada_val, y_ada_val_proba)

cm_ada_val = confusion_matrix(y_ada_val, y_ada_val_pred)

plt.figure(figsize=(4,3))
sns.heatmap(cm_ada_val, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix - AdaBoost (val)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(FIG / 'confusion_AdaBoost_val.png', dpi=200)
plt.close()

fpr, tpr, _ = roc_curve(y_ada_val, y_ada_val_proba)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC={ada_val_auc:.4f}')
plt.plot([0,1],[0,1],'--')
plt.title('ROC - AdaBoost (val)')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.tight_layout()
plt.savefig(FIG / 'roc_AdaBoost_val.png', dpi=200)
plt.close()

val_results.append({
    'model': 'AdaBoost',
    'accuracy': ada_val_acc,
    'precision': ada_val_prec,
    'recall': ada_val_rec,
    'f1': ada_val_f1,
    'roc_auc': ada_val_auc
})

# Test
y_ada_test_pred  = ada.predict(X_ada_test_scaled)
y_ada_test_proba = ada.predict_proba(X_ada_test_scaled)[:, 1]

ada_test_acc  = accuracy_score(y_test_full, y_ada_test_pred)
ada_test_prec = precision_score(y_test_full, y_ada_test_pred, zero_division=0)
ada_test_rec  = recall_score(y_test_full, y_ada_test_pred, zero_division=0)
ada_test_f1   = f1_score(y_test_full, y_ada_test_pred, zero_division=0)
ada_test_auc  = roc_auc_score(y_test_full, y_ada_test_proba)

cm_ada_test = confusion_matrix(y_test_full, y_ada_test_pred)

plt.figure(figsize=(4,3))
sns.heatmap(cm_ada_test, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix - AdaBoost (test)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig(FIG / 'confusion_AdaBoost_test.png', dpi=200)
plt.close()

fpr, tpr, _ = roc_curve(y_test_full, y_ada_test_proba)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC={ada_test_auc:.4f}')
plt.plot([0,1],[0,1],'--')
plt.title('ROC - AdaBoost (test)')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.tight_layout()
plt.savefig(FIG / 'roc_AdaBoost_test.png', dpi=200)
plt.close()

test_results.append({
    'model': 'AdaBoost',
    'accuracy': ada_test_acc,
    'precision': ada_test_prec,
    'recall': ada_test_rec,
    'f1': ada_test_f1,
    'roc_auc': ada_test_auc
})

print("Classification report (test) - AdaBoost :")
print(classification_report(y_test_full, y_ada_test_pred, digits=4))


# ========= 4) Sauvegarde des résultats globaux =========
val_df  = pd.DataFrame(val_results)
test_df = pd.DataFrame(test_results)

print("\n=== Résultats validation ===")
print(val_df)

print("\n=== Résultats test ===")
print(test_df)

val_df.to_csv(OUT / 'val_metrics.csv', index=False)
test_df.to_csv(OUT / 'test_metrics.csv', index=False)

with open(OUT / 'val_metrics.json', 'w') as f:
    json.dump(val_results, f, indent=2)
with open(OUT / 'test_metrics.json', 'w') as f:
    json.dump(test_results, f, indent=2)

print("\nMetrics and figures saved in:", OUT, "and", FIG)



=== RANDOM FOREST ===
Classification report (test) - RandomForest :
              precision    recall  f1-score   support

           0     0.3253    0.9949    0.4903     11457
           1     0.9992    0.7611    0.8641     98942

    accuracy                         0.7854    110399
   macro avg     0.6623    0.8780    0.6772    110399
weighted avg     0.9293    0.7854    0.8253    110399


=== KNN (avec PCA) ===
